## Notebook imports

In [3]:
import pandas as pd
import numpy as np

## Constants

In [106]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data-s.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data-s.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## Read and load Features from .txt Files into Numpy Array

In [5]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter= ' ', dtype=int)

In [7]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  5,  1,  1],
       [ 0, 10,  1,  2],
       [ 0, 47,  1,  1],
       [ 0, 75,  1,  1]])

In [8]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 266236
Nr of rows in test file 109917


In [9]:
print('Nr of emails in training file', np.unique(sparse_train_data[:,0]).size)

Nr of emails in training file 4016


### How to create an empty dataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[0:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5796, 5797, 5798])

In [12]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace = True)

In [13]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
sparse_train_data[10:13]

array([[   0,  568,    1,    1],
       [   0,  852,    1,    1],
       [   0, 1010,    1,    1]])

## Create A full matrix from a sparse matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    form a full matrix from a sparse matrix, return a pandas dataframe.
    keyword arguments:
    sparse matrix -- numpy array
    nr_words -- size of the vocabulary. total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of th label (spam is 1, nonspam is 0). Default 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix
    

In [16]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 7.59 s, sys: 109 ms, total: 7.7 s
Wall time: 7.69 s


In [17]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5792,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,2,0,1,1,1,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5796,0,1,0,4,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5797,0,1,0,2,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5798,0,1,2,1,0,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training The Naive Bayes Model

## calculating the probability of Spam

In [18]:
# challenge: calculate the probability of spam - the percent of spam messages in the training dataset. 
# store this value in a variable called prob_spam

In [19]:
type(full_train_data)

pandas.core.frame.DataFrame

In [20]:
full_train_data.CATEGORY.sum()

1250

In [21]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print("probability of spam is: ", prob_spam)

probability of spam is:  0.3112549800796813


## total number of words / Tokens

In [35]:
##loc is for selecting a number of rows
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY'] 
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,2,1,0,3,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,2,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,3,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5792,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,2,0,1,1,1,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5796,1,0,4,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
email_lengths = full_train_features.sum(axis=1) # axis =1 sums colums

In [26]:
total_ws = email_lengths.sum()
print(total_ws)

445901


## Numbers of tokens in spam & an ham emails

In [43]:
full_train_spam_features = full_train_data.loc[full_train_data['CATEGORY'] == 1, full_train_data.columns != 'CATEGORY']

In [42]:
spam_lengths = full_train_spam_features.sum(axis=1)

In [44]:
spam_lengths.shape
total_spam = spam_lengths.sum()

print(total_spam)

185054


In [67]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]

In [64]:

total_ham = ham_lengths.sum()
total_ham

260847

In [65]:
ham_lengths.shape

(2766,)

In [66]:
full_train_features.shape

(4016, 2500)

## Summing the tokens Occuring in Spam

In [72]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]


In [69]:
train_spam_tokens.head()


,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,2,1,0,3,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,2,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,3,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1 # laPlace smoothing
summed_spam_tokens.shape

(2500,)

In [81]:
# Challenge: Repeat this process for the ham messages. Sum the tokens that occur in the non-spam messages. Store the values in a avariable called summed_ham_tokens

## Summing the tokens occuring in ham

In [82]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis = 0) + 1 # laPlace smoothing

In [109]:
train_ham_tokens[2499].sum() + 1

26

## P(Token | Spam) - Probability that a token Occurs given the Email is Spam

In [85]:
prob_tokens_spam = summed_spam_tokens / (total_spam + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012082
1    0.004921
2    0.007326
3    0.010786
4    0.006814
dtype: float64

In [86]:
prob_tokens_spam.sum()

1.0

## P(token | Ham) - Probability that a tokaen occurs given the email is nonspam

In [87]:
prob_tokens_nonspam = summed_ham_tokens / (total_ham + VOCAB_SIZE) # adding vocab_size because of laPlace smootinh
prob_tokens_nonspam.sum()

1.0

## P(Token) - Probability that token Occurs

In [91]:
prob_tokens_all = full_train_features.sum(axis=0) / total_ws

In [92]:
prob_tokens_all.sum()

1.0

## Save the trained Model

In [99]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data

In [100]:
sparse_test_data.shape

(109917, 4)

In [101]:
%%time 
full_test_data = make_full_matrix(sparse_test_data, nr_words = VOCAB_SIZE)

CPU times: user 3.13 s, sys: 21 ms, total: 3.15 s
Wall time: 3.14 s


In [104]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [107]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)